<a href="https://colab.research.google.com/github/ShreyPandit/SAiDL-Winter-Assignment-2019/blob/master/One_Shot_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

In [0]:
batch_size = 16
num_epochs = 5

In [0]:
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size,
                                          shuffle=True)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





In [0]:
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size,
                                         shuffle=False)

In [0]:
iterator = iter(trainloader)
img = next(iterator)
img[0].shape

torch.Size([16, 1, 28, 28])

In [0]:
class Net(nn.Module):
  
    def __init__(self): 
        super(Net, self).__init__()
        self.cnn_model = nn.Sequential(
            nn.Conv2d(1, 6, 5),     #Can't quite figure out what's wrong    
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2),  
            nn.Conv2d(6, 16, 5),          
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2)   
        )
        self.fc = nn.Linear(256,10)
  
        
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [0]:
net = Net()

In [0]:
print(net)

Net(
  (cnn_model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=10, bias=True)
)


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [0]:
min_test_loss = 1000

In [0]:
model=net(inputs)
model.cuda()

tensor([[-0.0033,  0.1001, -0.3136,  ..., -0.1141, -0.1668, -0.0772],
        [ 0.1964, -0.0074, -0.1682,  ..., -0.1540, -0.1873, -0.0946],
        [ 0.0398,  0.0845, -0.2884,  ...,  0.0697, -0.1658, -0.0562],
        ...,
        [-0.0813,  0.0541, -0.2953,  ..., -0.1553, -0.2480,  0.0146],
        [ 0.1108,  0.0385, -0.2085,  ...,  0.1021, -0.1739, -0.0257],
        [-0.0203,  0.1777, -0.3854,  ..., -0.0560, -0.1751,  0.0720]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [0]:
for epoch in range(num_epochs):
    running_loss_train = 0.0
    running_loss_test = 0.0
    
    net.train()
    
    for i,data in enumerate(trainloader,0):

      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      output = net(inputs)
      loss = criterion(output, labels)

      loss.backward()
      optimizer.step()

      running_loss_train += loss.item()      

    net.eval()

    correct_preds = 0
    total_preds = 0
    
    for i,data in enumerate(testloader,0):

      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      output = net(inputs)
      loss = criterion(output,labels)


      running_loss_test += loss.item()

      _, pred = torch.max(output.data, 1)
      correct_preds += (pred == labels).sum()
      total_preds += labels.size(0)
        
    accuracy = 100.00 * float(correct_preds)/float(total_preds)
    

        
    print(f'Epoch : {epoch+1}, Train Loss : {running_loss_train} Test Loss : {running_loss_test}, Accuracy : {accuracy}')

    
    if running_loss_test<min_test_loss:
        torch.save(net.state_dict(), 'mnist.pt')
        min_test_loss = running_loss_test
 

TypeError: ignored

In [0]:
net.load_state_dict(torch.load('mnist.pt'))


In [0]:
dataset = datasets.Omniglot(root = "./data", download = True, transform = transforms.ToTensor())


Extracting ./data/omniglot-py/images_background.zip to ./data/omniglot-py



In [0]:
len(dataset)

19280

In [0]:
class SiameseNetworkDataset(Dataset):
   
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))

    def __len__(self):
        return len(self.imageFolderDataset.imgs)        

In [0]:
folder_dataset = datasets.ImageFolder(root = '/content/data/omniglot-py')


In [0]:
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transforms.Compose([transforms.Resize((28,28)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

In [0]:
class SiameseNetwork(nn.Module):
    def __init__(self, model):
        super(SiameseNetwork, self).__init__()
        
        self.model = model
        self.fc = nn.Linear(256,5)
    
    def forward_once(self, x):
        out = self.model(x)
        out = F.relu(self.fc(out))
        return out
    
    def forward(self, input1, input2):
        out1 = self.forward_once(input1)
        out2 = self.forward_once(input2)
        return out1,out2

In [0]:
class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [0]:
train_loader = torch.utils.data.DataLoader(siamese_dataset,
                                         shuffle=True,
                                         batch_size=16)

In [0]:
img = next(iter(train_loader))
img[0].shape

torch.Size([16, 1, 28, 28])

In [0]:
net.fc = nn.Linear(256,256)
print(net)

Net(
  (cnn_model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=256, bias=True)
)


In [0]:
SiamNet = SiameseNetwork(net)


In [0]:
print(SiamNet)

SiameseNetwork(
  (model): Net(
    (cnn_model): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
      (1): Tanh()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
      (4): Tanh()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc): Linear(in_features=256, out_features=256, bias=True)
  )
  (fc): Linear(in_features=256, out_features=5, bias=True)
)


In [0]:
criterion = ContrastiveLoss()
opt = optim.Adam(SiamNet.parameters(),lr = 0.001)

In [0]:
model=SiamNet
model.cuda()

SiameseNetwork(
  (model): Net(
    (cnn_model): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
      (1): Tanh()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
      (4): Tanh()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc): Linear(in_features=256, out_features=256, bias=True)
  )
  (fc): Linear(in_features=256, out_features=5, bias=True)
)

In [0]:
for epoch in range(10):
    
    for img1, img2, label in train_loader:

        img1, img2, label = img1.to(device), img2.to(device), label.to(device)

        opt.zero_grad()

        output1, output2 = SiamNet(img1, img2)

        loss = criterion(output1, output2, label)
        loss.backward()

        opt.step()
    
    print(f'Epoch {epoch+1}  Loss = {loss.item()}')

Epoch 1  Loss = 4.999999980020986e-12
Epoch 2  Loss = 4.999999980020986e-12
Epoch 3  Loss = 4.999999980020986e-12
Epoch 4  Loss = 4.999999980020986e-12
Epoch 5  Loss = 4.999999980020986e-12
Epoch 6  Loss = 4.999999980020986e-12
Epoch 7  Loss = 4.999999980020986e-12
Epoch 8  Loss = 4.999999980020986e-12
Epoch 9  Loss = 4.999999980020986e-12
Epoch 10  Loss = 4.999999980020986e-12
